## Parser dos dados de Acelerômetro (Ximango e CO)

Do ximango são extraídos os dados de acelerômetro de todos veículos ativos (.csv), não há informação do Ximango sobre data/hora de início/fim de viagem, veículo ou linha.
Os dados das viagens realizadas são extraídos do CO, nesse caso é um período específico por cada veículo.

OBS: No primeiro momento está sendo importado somente dados de ximango e co referente a uma linha específica. (Linha 360)

### 1. Lendo dados Ximango

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib as plt

In [ ]:
df = pd.read_csv('/home/rogerio/Teste-Linha360.csv', 
                 sep=r'\,|\;',
                 header=None,
                 usecols=[1,3,4,5,6,7,9,10,11,12,13,14,15,16],
                 engine='python')
df.rename(columns={1:"time", 
                   3:"yn", 
                   4:"lat_yn", 
                   5:"lon_yn", 
                   6:"yp", 
                   7:"lat_yp", 
                   9:"lon_yp", 
                   10:"xn", 
                   11:"lat_xn", 
                   12:"lon_xn", 
                   13:"xp", 
                   14:"lat_xp", 
                   15:"lon_xp", 
                   16:"id"}, 
          inplace=True)
df['time'] = pd.to_datetime(df.time,format='%d%m%y%H%M%S')
df['time'] = df.time + pd.to_timedelta(-2,unit='h')
df['yn'] = df.yn.apply(lambda x: 4294967000+x-4294967296)
df['xn'] = df.xn.apply(lambda x: 4294967000+x-4294967296)
df['id'] = pd.Series(df['id']).str.replace('ID=', '', regex=True)
df['id'] = pd.to_numeric(df['id'])

df.head(2)

### 2. Lendo dados  CO

In [ ]:
dfco = pd.read_csv('/home/rogerio/Teste-Linha360CO.csv',
                   header=None,
                   usecols=[0,1,2,3,4,5],
                   engine='python')
dfco.rename(columns={0:"id_v",
                     1:"id",
                     2:"trip_time",
                     3:"line",
                     4:"pc",
                     5:"sentido"},
            inplace=True)
dfco['trip_time'] = pd.to_datetime(dfco.trip_time,format='%d/%m/%y %H:%M:%S')

dfco.head(2)

### 3. Manipulando os dados
#### 3.1 Ximango
Separar os eventos por id (chave externa, virloc)
Para cada id usar os dados de viagem (dt inicio e data fim) do CO para separar as viagens contindas em cada ID.

In [ ]:
# remove lat, long (nesse momento não preisa)
df = df[['time', 'yn', 'yp', 'xn', 'xp', 'id']]

# divisão do df em IDs
# minha key será o id do df e o valor é o df.
g = df.groupby('id')

# for g_id, g_id_df in g:
#     # g_id: key 
#     # g_id_df: valores (dataframe separado de cada key)
#     display(g_id)
#     display(g_id_df.head(2))
    
#df.groupby('id').mean()
#df.groupby('id').xp.agg(['count', 'min', 'max','mean', 'std'])

#### 3.2 Estatística de todos os eventos da Linha 360, separado por veículos.

Tabela:

In [ ]:
g.agg(['max','min','mean','std'])

#### Gráfico do Desvio Padrão de todas as viagens (separados por veículo)

In [ ]:
a = g.agg(['std'])
a.reset_index(inplace=True)
a_plot = a.set_index('id').T
a_plot.plot.bar(figsize=(15,9), xticks=range(0, 8), grid=True).legend(title='Desvio Padrão', bbox_to_anchor=(1, 1))


#### Gráfico da Média de todas as eventos (separados por veículo)

In [ ]:
a = g.agg(['mean'])
a.reset_index(inplace=True)
a_plot = a.set_index('id').T
a_plot.plot.bar(figsize=(15,9), grid=True).legend(title='Média', bbox_to_anchor=(1, 1))


#### 3.2 CO
Obter as viagens com data inicio/fim e id veículo.

In [ ]:
gco = dfco.groupby('id')

# for gco_id, gco_id_df in gco:
#     # gco_id: key 
#     # gco_id_df: valores (dataframe separado das viagens com cada id veic)
#     display(gco_id)
#     display(gco_id_df)

DataFrame gco está agrupado por id.
Cada id (veículo) possui todas as viagens do período selecionado.
Considerando que <font color='red'>as viagens selecionadas (.csv co) não possuem problema de duplicidade de horário, pc e já estão com índice <b>i</b>  para inicio e índice  <b>i+1</b> para fim</font> o parser cria uma máscara para agrupar os eventos do ximango de acordo com essas viagens.


In [ ]:
df_trip = pd.DataFrame({'id_v':[], 'id':[], 'start':[], 'end':[]})

# em cada veículo
for gco_id, gco_id_df in gco:
    # em cada evento (viagens de cada veículo)
    for i in range(0, len(gco_id_df)-1, 2):
        start_date = gco_id_df.iloc[i]['trip_time']
        end_date = gco_id_df.iloc[i+1]['trip_time']
        id_v = gco_id_df.iloc[i]['id_v']
        df_trip = df_trip.append({'id_v':id_v, 'id':gco_id, 'start':start_date, 'end':end_date}, ignore_index=True)
        i = i + 2

Dataframe df_trip contém o início e final de cada viagem.

In [ ]:
df_trip['id'] = df_trip['id'].astype(np.int64)
df_trip['id_v'] = df_trip['id_v'].astype(np.int64)

df_trip.head(10)

### 4. Visualização e Resultados

#### 4.1 Eventos de acelerômetro separado por viagem

In [ ]:
count = 0
for i in range(len(df_trip)):
    start_date = df_trip.iloc[i]['start']
    end_date = df_trip.iloc[i]['end']
    mask = (df['time'] > start_date ) & (df['time'] < end_date )
    df_vg = df.loc[mask]
    if df_vg.count()['yn'] != 0:
        display("Veículo: {}/{} || Dur: {} || Início: {} Fim: {}" .format(df_trip.iloc[i]['id'],
                                                                      df_trip.iloc[i]['id_v'],
                                                                      (df_trip.iloc[i]['end'] - df_trip.iloc[i]['start']),           
                                                                      df_trip.iloc[i]['start'],
                                                                      df_trip.iloc[i]['end']))
        display(df_vg)
    count = count + 1


In [ ]:
# # teste de lista de dataframes
# df_eventos = []
# count = 0
# for i in range(len(df_trip)):
#     start_date = df_trip.iloc[i]['start']
#     end_date = df_trip.iloc[i]['end']
#     mask = (df['time'] > start_date ) & (df['time'] < end_date )
#     df_vg = df.loc[mask]
#     if df_vg.count()['yn'] != 0:
#         df_eventos.append(df_vg)
#     count = count + 1

#### 4.2 Estatística de cada Viagem

In [ ]:
count = 0
for i in range(len(df_trip)):
    start_date = df_trip.iloc[i]['start']
    end_date = df_trip.iloc[i]['end']
    mask = (df['time'] > start_date ) & (df['time'] < end_date )
    df_vg = df.loc[mask]
    if df_vg.count()['yn'] != 0:    
        display("Veículo: {}/{} || Dur: {} || Início: {} Fim: {}" .format(df_trip.iloc[i]['id'],
                                                                      df_trip.iloc[i]['id_v'],
                                                                      (df_trip.iloc[i]['end'] - df_trip.iloc[i]['start']),           
                                                                      df_trip.iloc[i]['start'],
                                                                      df_trip.iloc[i]['end']))
        display(df_vg.agg(['count', 'min', 'max','mean', 'std']))
    count = count + 1

#### 4.3 Gráficos de cada Viagem

In [ ]:
# quando ultrapassa de 20 gráficos é exibido um aviso que pode consumir muita memória
plt.rcParams.update({'figure.max_open_warning': 0})

%matplotlib inline
for i in range(len(df_trip)):
    start_date = df_trip.iloc[i]['start']
    end_date = df_trip.iloc[i]['end']
    mask = (df['time'] > start_date ) & (df['time'] < end_date )
    df_vg = df.loc[mask]
    if df_vg.count()['yn'] != 0:
        df_vg.plot('time',['xp','yp','xn','yn'], marker='o', figsize=(15,10), grid=True).legend(
            title=str(df_trip.iloc[i]['id'])+' // '+str(df_trip.iloc[i]['start']), bbox_to_anchor=(1,1))


#### 4.4 Gráficos Estatística de cada Viagem

In [ ]:
count = 0
for i in range(len(df_trip)):
    start_date = df_trip.iloc[i]['start']
    end_date = df_trip.iloc[i]['end']
    mask = (df['time'] > start_date ) & (df['time'] < end_date )
    df_vg = df.loc[mask]
    if df_vg.count()['yn'] != 0:    
        b = df_vg.agg(['mean', 'std'])
        b[['yn','yp','xn','xp']].plot.barh(figsize=(12,8), grid=True).legend(title=str(df_trip.iloc[i]['id'])+' // '+str(df_trip.iloc[i]['start']), bbox_to_anchor=(1,1))
    count = count + 1

# figsize=(15,10), grid=True).legend(
# title=str(df_trip.iloc[i]['id'])+' // '+str(df_trip.iloc[i]['start']), bbox_to_anchor=(1,1))
